# A tour of risk and performance

In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append('../')

## Introduction

## Alpha and Beta

##  Where does alpha come from 

In [4]:
import yfinance as yf
import pandas as pd

# Define the ticker symbols for Synchrony Financial, Walmart, and S&P 500
tickers = ['SYF', 'WMT', '^GSPC', 'SPY']

# Download the daily prices for the specified tickers
data = yf.download(tickers, start='2010-01-01', end='2024-12-31')

# Extract the 'Adj Close' prices
adj_close = data['Adj Close']

# Compute the daily returns
daily_returns = adj_close.pct_change().dropna()

# Display the first few rows of the daily returns
print(daily_returns.head())
 


[*********************100%%**********************]  4 of 4 completed
                 SPY       SYF       WMT     ^GSPC
Date                                              
2014-08-01 -0.003056  0.000000 -0.000544 -0.002859
2014-08-04  0.007221  0.000000  0.000000  0.007189
2014-08-05 -0.009697  0.000000 -0.002719 -0.009685
2014-08-06  0.000313 -0.000870  0.018391  0.000016
2014-08-07 -0.005415 -0.002176 -0.003369 -0.005557


In [5]:
adj_close.reset_index(inplace=True)

In [6]:
adj_close['Date']= pd.to_datetime(adj_close['Date']).dt.date

In [7]:
adj_close['year']=  adj_close['Date'].apply(lambda d: d.year)

In [8]:
adj_close.sort_values(by='Date', ascending = True, inplace = True)

In [9]:
adj_close['SYF_prior_1D_return'] = adj_close['SYF']/adj_close['SYF'].shift(1)-1

In [10]:
adj_close['WMT_prior_1D_return'] = adj_close['WMT']/adj_close['WMT'].shift(1)-1

In [11]:
adj_close['S&P_prior_1D_return'] = adj_close['^GSPC']/adj_close['^GSPC'].shift(1)-1
adj_close['SPY_prior_1D_return'] = adj_close['SPY']/adj_close['SPY'].shift(1)-1

In [12]:
import statsmodels.api as sm

In [13]:
for y, data in adj_close.groupby('year'):
    
    Xy =  data[['SYF_prior_1D_return' , 'SPY_prior_1D_return']].dropna()
    if Xy.shape[0]>30:
        print(y)
        model=sm.OLS( Xy['SYF_prior_1D_return'], sm.add_constant(Xy['SPY_prior_1D_return'])  ).fit()
        alpha_daily = model.params['const']
        trading_days_per_year = 252  # Approximate number of trading days in a year
        alpha_annualized = (1 + alpha_daily) ** trading_days_per_year - 1
        alpha_conf_int = model.conf_int().loc['const']

        
        print( f'Annualized alpha {alpha_annualized} in {y} ')

2014
Annualized alpha 0.6452261550641496 in 2014 
2015
Annualized alpha 0.0354865444375998 in 2015 
2016
Annualized alpha 0.05657256941493949 in 2016 
2017
Annualized alpha -0.21116113432341788 in 2017 
2018
Annualized alpha -0.329384530717459 in 2018 
2019
Annualized alpha 0.1707044817862402 in 2019 
2020
Annualized alpha -0.09162056766502613 in 2020 
2021
Annualized alpha -0.007780848448860178 in 2021 
2022
Annualized alpha -0.004354982907281757 in 2022 
2023
Annualized alpha -0.09778389768101492 in 2023 
2024
Annualized alpha 0.3333645436598607 in 2024 


In [15]:
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
======================================================================
Model:              OLS                 Adj. R-squared:     0.061     
Dependent Variable: SYF_prior_1D_return AIC:                -646.2618 
Date:               2024-06-30 09:21    BIC:                -640.6212 
No. Observations:   124                 Log-Likelihood:     325.13    
Df Model:           1                   F-statistic:        8.965     
Df Residuals:       122                 Prob (F-statistic): 0.00333   
R-squared:          0.068               Scale:              0.00031414
-----------------------------------------------------------------------
                      Coef.   Std.Err.    t     P>|t|    [0.025  0.975]
-----------------------------------------------------------------------
const                 0.0011    0.0016  0.7071  0.4808  -0.0021  0.0043
SPY_prior_1D_return   0.7100    0.2371  2.9942  0.0033   0.2406  1.1795
----------------------------------------------------------------------
Omnibus:                9.692         Durbin-Watson:            2.039 
Prob(Omnibus):          0.008         Jarque-Bera (JB):         11.713
Skew:                   0.473         Prob(JB):                 0.003 
Kurtosis:               4.171         Condition No.:            149   
======================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is
correctly specified.
"""

## Estimate Risk in Advance

### What is Risk 

In [18]:
import scipy.stats as stats

def compute_prob_rare_event(num_std_dev):
    """
    Compute the probability of a rare event defined by the number of standard deviations from the mean
    in a normal distribution.
    
    Parameters:
    num_std_dev (float): The number of standard deviations from the mean that defines the rare event.
    
    Returns:
    float: The probability of the rare event.
    """
    # Calculate the probability of the event being within the specified number of standard deviations
    # prob_within_num_std_dev = stats.norm.cdf(num_std_dev) - stats.norm.cdf(-num_std_dev)
    
    # The probability of a rare event (outside the specified number of standard deviations)
    prob_rare_event = stats.norm.cdf(-num_std_dev)
    
    return prob_rare_event

# Number of standard deviations defining the rare event (you can adjust this value if needed)
for num_std_dev in [ 1.0, 2,2.5,3]:
    print(num_std_dev)

    # Compute the probability of a rare event
    prob_rare_event = compute_prob_rare_event(num_std_dev)

    # Number of days in a year
    total_days_per_year = 365

    # Number of business days in a year (approximation)
    business_days_per_year = 252

    # Number of years for extended calculation
    years = 5


    # Expected number of rare events for business days in a year
    expected_events_business_days_per_year = business_days_per_year * prob_rare_event
    # Expected number of rare events for business days in five years
    expected_events_business_days_five_years = expected_events_business_days_per_year * years

    # Output the results
    print(f"Probability of a rare event defined by {num_std_dev} standard deviation(s): {prob_rare_event:.5f}")
    print(f"Expected number of rare events in a year with business days: {expected_events_business_days_per_year:.2f}")
    print(f"Expected number of rare events in five years with business days: {expected_events_business_days_five_years:.2f}")


1.0
Probability of a rare event defined by 1.0 standard deviation(s): 0.15866
Expected number of rare events in a year with business days: 39.98
Expected number of rare events in five years with business days: 199.91
2
Probability of a rare event defined by 2 standard deviation(s): 0.02275
Expected number of rare events in a year with business days: 5.73
Expected number of rare events in five years with business days: 28.67
2.5
Probability of a rare event defined by 2.5 standard deviation(s): 0.00621
Expected number of rare events in a year with business days: 1.56
Expected number of rare events in five years with business days: 7.82
3
Probability of a rare event defined by 3 standard deviation(s): 0.00135
Expected number of rare events in a year with business days: 0.34
Expected number of rare events in five years with business days: 1.70


### Measuring Risk and Performance 

0.7100383266800439

In [19]:
trading_days_per_year = 252  # Approximate number of trading days in a year

net_market_value = {'SYF': 10_000_000,
                   'SPY': 10_000_000,
                   'WMT': 5_000_000}
for y, data in adj_close.groupby('year'):
    
    Xy =  data[['SYF_prior_1D_return' , 'SPY_prior_1D_return']].dropna()
    if Xy.shape[0]>30:
        print(y)
        model=sm.OLS( Xy['SYF_prior_1D_return'], sm.add_constant(Xy['SPY_prior_1D_return'])  ).fit()
        
        alpha_daily = model.params['const']
        beta = model.params['SPY_prior_1D_return']
        daily_market_volatility = Xy['SPY_prior_1D_return'].std()
        daily_idiosyncratic_volatility = model.resid.std()
        alpha_annualized = (1 + alpha_daily) ** trading_days_per_year - 1
        alpha_conf_int = model.conf_int().loc['const']
        
        dollar_market
        
        
        print( f'Annualized alpha {alpha_annualized} in {y} ')

2014
Annualized alpha 0.6452261550641496 in 2014 
2015
Annualized alpha 0.0354865444375998 in 2015 
2016
Annualized alpha 0.05657256941493949 in 2016 
2017
Annualized alpha -0.21116113432341788 in 2017 
2018
Annualized alpha -0.329384530717459 in 2018 
2019
Annualized alpha 0.1707044817862402 in 2019 
2020
Annualized alpha -0.09162056766502613 in 2020 
2021
Annualized alpha -0.007780848448860178 in 2021 
2022
Annualized alpha -0.004354982907281757 in 2022 
2023
Annualized alpha -0.09778389768101492 in 2023 
2024
Annualized alpha 0.3333645436598607 in 2024 


In [22]:
errors = Xy['SYF_prior_1D_return'] - model.predict(sm.add_constant(Xy['SPY_prior_1D_return']) )

In [24]:
errors - model.resid

3522    0.0
3523    0.0
3524    0.0
3525    0.0
3526    0.0
       ... 
3641    0.0
3642    0.0
3643    0.0
3644    0.0
3645    0.0
Length: 124, dtype: float64

3522   -0.003191
3523   -0.008779
3524    0.000878
3525    0.034744
3526   -0.001490
          ...   
3641    0.024777
3642   -0.011785
3643   -0.008229
3644   -0.016745
3645    0.068575
Length: 124, dtype: float64

## First steps in risk decomposition 

## Simple Hedging 

## Separation of concerns 

## Takeaways 